# Problem Statement:

We want to train a simple feedforward neural network to calculate terminal velocity of a solid sphere of density $\sigma$ and radius r, in a liquid of density $\rho$ and viscosity $\eta$.

The formula for this terminal velocity classically is :
$$V_{t} = \frac{2r^{2}(\rho - \sigma)g}{9\eta}$$

Thus there will be 4 inputs into the neural network, densities $\sigma$ and $\rho$, viscosity $\eta$ and radius r. And there will be one output corresponding to the terminal velocity.

We will use above equation to simulate and create data to put into our neural network.


### The twist

We are only allowed to implement this neural network completely from scratch without any external libraries
Only python and numpy allowed (and Matplotlib for plotting purposes)

Additionally, I can only reference the 3b1b deep learning series for help, and chatgpt assisstance is limited to only conceptual doubts.

In [24]:
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Creating the Data 

Chatgpt suggested these range of values for simulating the data

- > r from 10^-4 to 5 * 10^-3 m
- > $\sigma$ from 500 to 8000 kg/m^3
- > $\rho$ from 700 to 1300 kg/m^3
- > $\eta$ from 0.001 to 2 Pa $\cdot$ s

We will consider 5k input data points into the neural network

In [25]:
dataPoints = 5000

In [26]:
rng = np.random.default_rng(seed = 42)

r = rng.uniform(low = 0.0001, high = 0.005, size = (dataPoints, 1))
densSolid = rng.uniform(low = 500, high = 8000, size = (dataPoints, 1))
densLiquid = rng.uniform(low = 700, high = 1300, size = (dataPoints, 1))
viscosity = rng.uniform(low = 0.001, high = 2, size = (dataPoints, 1))

data = np.hstack((r, densSolid, densLiquid, viscosity))

In [27]:
def terminal(dats):
    r, densL, densS, visc = dats
    return (2*(r**2)*(densL - densS)*9.8)/(9*visc)

v = np.apply_along_axis(terminal, 1, data)

## Step 2: Train/test split and Scaling the data

We shall do a 80/20 standard split

We will use Z-score scaling or standardization to scale
We will learn the means and stds from training data and apply same on testing data

We will scale both inputs and outputs since thats necessary for neural networks, and we will unscale the outputs at the end

In [28]:
index = int((0.8) * data.shape[0])

xTrain = data[:index, :]
xTest = data[index: , :]
yTrain = v[:index]
yTest = v[index:]

In [29]:
#Scaling inputs
xMean = np.mean(xTrain, axis = 0)[np.newaxis, :]
xStd = np.std(xTrain, axis = 0)[np.newaxis, :]

xTrain = (xTrain - xMean)/xStd
xTest = (xTest - xMean)/xStd

#Scaling outputs
yMean = np.mean(yTrain)
yStd = np.std(yTrain)
yTestOrig = yTest
yTrainOrig = yTrain

yTrain = (yTrain - yMean)/yStd
yTest = (yTest - yMean)/yStd

In [30]:
yScalingParams = np.array([yMean, yStd])

np.save("./data/xTrain.npy", xTrain)
np.save("./data/xTest.npy", xTest)
np.save("./data/yTrain.npy", yTrain)
np.save("./data/yTest.npy", yTest)
np.save("./data/yTestOrig.npy", yTestOrig)
np.save("./data/yTrainOrig.npy", yTrainOrig)
np.save("./data/yScalingParams.npy", yScalingParams)